#### Задача 1 (10 баллов). 

Напишите программу - базу данных для ветеринарной клиники (мы что-то подобное уже делали, если есть старый код - можно его доработать). В базе данных (можете для хранения смело использовать pickle) хранятся записи о питомцах, их хозяевах и даты записи на прием, а также комментарии ветеринара, если прием уже состоялся. Таким образом, нам понадобятся:

- класс "клиент", в котором будет храниться имя и контакты хозяина, а также его питомцы. 
- класс "питомец", в котором будут храниться имя и возраст питомца, даты записи на прием (можно реализовать как меняющийся атрибут "текущий прием", если питомца хотят привести, и атрибут-список или словарь "приемы", куда будут добавляться прошедшие приемы). 
- общий класс "база", где будут храниться клиенты. По базе можно осуществлять поиск клиента по имени или по номеру телефона. 
- дескрипторы / property, чтобы нельзя было вместо имени записать чепуху, указать питомцу возраст недопустимого значения и можно еще телефон проверять. 
- в основном коде программы должна быть возможность посмотреть записи о прошедших приемах, наличие записи на текущий прием; возможность отыскать клиента и установить для одного из его питомцев (видимо, понадобится уметь выводить список питомцев) дату нового приема, а также отметить, что прием уже состоялся, и внести комментарий ветеринара.
- Ну и, конечно, базу данных можно сохранять. Можно это делать при выборе "выйти из программы". 

    Эту задачу можно написать в тетрадке, если хочется, но, возможно, будет удобнее в py. 

In [1]:
import pickle
import re

In [10]:
class Name:
    '''descriptor class for names'''
    def __get__(self, pet, type=None):
        if hasattr(pet, 'nam'):
            return pet.nam
        raise AttributeError
    
    def __set__(self, pet, value):
        if value.isalpha():
            pet.nam = value
        else:
            print('Can\'t set! Names are made of letters!!')


class Age:
    '''descriptor for pet ages'''
    def __get__(self, pet, type=None):
        if hasattr(pet, 'ag'):
            return pet.ag
        raise AttributeError
    
    def __set__(self, pet, value):
        if isinstance(value, int) and 0 <= value <= 50: # для черепах)
            pet.ag = value
        else:
            print('Can\'t set! Appropriate age is from 0 to 50')


class Number:
    '''phone number descriptor'''
    def __get__(self, petowner, type=None):
        if hasattr(petowner, 'num'):
            return petowner.num
        raise AttributeError
    
    def __set__(self, petowner, value):
        if re.fullmatch(r'^7\d{10}$', value): # в формате 7XXXXXXXXXX
            petowner.num = value
        else:
            print('Can\'t set! Phone number format is 7XXXXXXXXXX')


class Client:
    name = Name()
    contactnumber = Number()
    def __init__(self, name, contactnumber):
        self.name = name
        self.contactnumber = contactnumber
        self.pets = []

    # добавить клиенту питомцев
    def addnewpet(self, pet):
        self.pets.append(pet)

    # вернуть всех питомцев
    def getallpets(self):
        return self.pets

    def __str__(self):
        return f'Client: {self.name}, phone number: {self.number}, pets: {[pet.name for pet in self.pets]}'                


class Pet:
    name = Name()
    age = Age()
    def __init__(self, name, age):
        self.name = name
        self.age = age
        self.appointment = None
        self.allappnts = [] # все приемы

    # добавить запись
    def addappointment(self, date):
        self.appointment = date

    # закончить приём
    def endingappnt(self, comment):
        if self.appointment:
            self.allappnts.append({'date': self.appointment, 'vet comment': comment})
            self.appointment = None
        else:
            print('There\'s no appointment happening')

    # добавить комм к прошлому приёму
    def commentonthepast(self, date, comment):
        for appnt in self.allappnts:
            if appnt['date'] == date:
                appnt['comment'] = comment
                return True
        return False

    # вернуть все приёмы питомца
    def getallappnts(self):
        return self.allappnts       


class Database:
    def __init__(self):
        self.clients = []

    @classmethod
    def loaddb(cls, filename='vetclinic_db.pkl'): # загрузить базу
        try:
            with open (filename, 'rb') as file:
                return pickle.load(file)
        except FileNotFoundError:
            #print('There\'s no such file :(')
            return cls()         

    def save(self, filename='vetclinic_db.pkl'): # сохранить базу
        with open(filename, 'wb') as file:
            pickle.dump(self, file)
        print('\nYour db is saved!!')   

    # добавить клиента в базу
    def addnewclient(self, client): 
        self.clients.append(client)

    # теперь поищем клиента
    def findclientsname(self, name):
        for client in self.clients:
            if client.name == name:
                return client
            
    # поищем по номеру
    def findclientsnumber(self, number):
        for client in self.clients:
            if client.number == number:
                return client
            
    def showappts(self): # shows all appointments
        for client in self.clients:
            for pet in client.pets:
                for appointment in pet.allappnts:
                    print(f'Pet: {pet.name}, appointment date: {appointment.date}, vet comments: {appointment.vetcomments}')        
                

class ClinicProgram:
    '''Класс программы для ветбазы'''
    def __init__(self):
        self.db = Database.loaddb()   

    def addclients(self): # добавить клиента в базу
        name = input('Client name: ')
        number = input('Client\'s phone number (7XXXXXXXXXX):')
        try:
            client = Client(name, number)
            self.db.addnewclient(client)
            print('\nNew client was added to the DB')
        except ValueError as error:
            print('Please try again with normal name and number...')
            print(error)

    def clientsbynames(self): # поиск клиента по имени
        name = input('What name should I search in the DB? ')
        client = self.db.findclientsname(name)
        if client:
            print(client)
        else:
            print('\nThere\'s no client by that name yet')

    def clientsbynumber(self): # поиск клиента по номеру
        number = input('What phone number should I search in the DB? ')
        client = self.db.findclientsnumber(number)
        if client:
            print('\n')
            print(client)
        else:
            print('\nThere\'s no client with that phone number yet') 

    def clientpluspet(self): # добавляем питомца к клиенту
        number = input('Enter cliet\'s phone number to add a pet: ')
        client = self.db.findclientsnumber(number)
        if client:
            petname = input('Pet\'s name: ')
            try:
                petage = int(input('Pet\'s age: '))
                pet = Pet(petname, petage)
                client.addnewpet(pet)
                print(f'\nPet {petname} was added to client {client.name}.')
            except ValueError as error:
                print(error)
        else:
            print('\nNo such client in the DB')                   

    # будущие приёмы
    def addfutureappointment(self):
        number = input('Enter cliet\'s phone number to add a future appointment to a pet: ')
        client = self.db.findclientsnumber(number)
        if client:
            petname = input('Pet\'s name: ')
            # смотрим сквозь всех питомцев клиента быстро
            pet = next((x for x in client.getallpets() if x.name == petname), None)
            if pet:
                date = input('Enter appointment date (DD-MM-YYYY): ')
                pet.addappointment(date)
                print(f'\nA pet {pet.name} has a new appointment on {date}.')
            else:
                print('\npet not found')
        else:
            print('\nClient not found')

    # закончить приём
    def endappointment(self):
        number = input('Enter cliet\'s phone number to end the appointment: ')
        client = self.db.findclientsnumber(number)
        if client:
            petname = input('Pet\'s name: ')
            pet = next((x for x in client.getallpets() if x.name == petname), None)
            if pet:
                if pet.appointment:
                    comment = input('Vet comment: ')
                    pet.endingappnt(comment)
                    print(f'\n{pet.name}\'s appointment is over, vet has recommended: {comment}')
                else:
                    print('\nno appointment at the moment')
            else:
                print('\npet not found')
        else:
            print('\nClient not found')

    # показать прошлые приёмы
    def showpreviousappnts(self):
        number = input('Enter cliet\'s phone number to end the appointment: ')
        client = self.db.findclientsnumber(number)
        if client:
            petname = input('Pet\'s name: ')
            pet = next((x for x in client.getallpets() if x.name == petname), None)
            if pet:
                previousappnts = pet.getallappnts()
                if previousappnts:
                    print(f'\n{pet.name}\'s previous appointments: ')
                    for appnt in previousappnts:
                        print(f"\n- Date: {appnt['date']}, vet comments: {appnt.get('comment')}")
                else:
                    print('\nThis pets doesn\'t have any previous appointments')
            else:
                print('\npet not found')
        else:
            print('\nClient not found')

    def reminiscence(self): # комменты врача к предыдущим приёмам добавить
        number = input('Enter cliet\'s phone number to end the appointment: ')
        client = self.db.findclientsnumber(number)
        if client:
            petname = input('Pet\'s name: ')
            pet = next((x for x in client.getallpets() if x.name == petname), None)
            if pet:
                date = input('Enter DD-MM-YYYY of the previous appointment: ')
                comment = input('New comment: ')
                if pet.commentonthepast(date, comment):
                    print(f'\nNew comment added to the {date} appointment')
                else:
                    print(f'\n{date} appointment not found')
            else:
                print('\npet not found')
        else:
            print('\nClient not found')

    # вывести клиентов
    def showallclients(self):
        for client in self.db.clients:
            print(client)

    # выйти из базы и сохранить
    def saveandexitdb(self):
        self.db.save()
        print('\nAll changes are saved. Exiting the program now...')
        #quit()

    # то, что всё запускает
    def operate(self):
        '''открывает программу и запускает ВСЁ'''
        print('\n1 - add new client')
        print('2 - find a client by their name ')
        print('3 - find a client by their phone number')
        print('4 - add a pet to the client')
        print('5 - nem appointment for a pet')
        print('6 - end an appointment')
        print('7 - look through pet\'s past appointments')
        print('8 - add a comment to a finished appointment')
        print('9 - show all clients')
        print('10 - SAVE AND EXIT')

        while True:
            choice = input('\nChoose an action: ')
            if choice == '1':
                self.addclients()
            elif choice == '2':
                self.clientsbynames()
            elif choice == '3':
                self.clientsbynumber()
            elif choice == '4':
                self.clientpluspet()
            elif choice == '5':
                self.addfutureappointment()
            elif choice == '6':
                self.endappointment()
            elif choice == '7':
                self.showpreviousappnts()
            elif choice == '8':
                self.reminiscence()
            elif choice == '9':
                self.showallclients()
            elif choice == '10':
                self.saveandexitdb()
                break
            else:
                print('Weird choice, try again...')


In [ ]:
# CharliXCX 78005553535 Шарик 10
# Клава 79155559409 Кока 49
# 13-09-2024 25-05-2024 19-02-2025

In [9]:
if __name__ == '__main__':
    program = ClinicProgram()
    program.operate()


1 - add new client
2 - find a client by their name 
3 - find a client by their phone number
4 - add a pet to the client
5 - nem appointment for a pet
6 - end an appointment
7 - look through pet's past appointments
8 - add a comment to a finished appointment
9 - show all clients
10 - SAVE AND EXIT

New client was added to the DB

Pet Кока was added to client Клава.

A pet Кока has a new appointment on 19-02-2025.

Кока's appointment is over, vet has recommended: жить дальше

New client was added to the DB
Client: Клава, phone number: 79155559409, pets: ['Кока']
Client: CharliXCX, phone number: 78005553535, pets: []

Your db is saved!!

All changes are saved. Exiting the program now...
